In [ ]:
import jax


In [ ]:
def make_HiPPO(N, HiPPO_type="legs"):
    mat = []
    for k in range(1, N + 1):
        for n in range(1, N + 1):
            if HiPPO_type == "legt":
                mat = build_LegT(n, k)
                # TODO
            elif HiPPO_type == "lagt":
                mat = build_LagT(alpha, beta, N)
                # TODO
            elif HiPPO_type == "legs":
                mat = build_LegS(n, k)
                # TODO
            elif HiPPO_type == "fourier":
                mat = build_FRU(N, theta=0.5)
                # TODO
            elif HiPPO_type == "random":
                A = np.random.randn(N, N) / N
                B = np.random.randn(N, 1)
                #TODO
            elif HiPPO_type == "diagonal":
                A = -np.diag(np.exp(np.random.randn(N)))
                B = np.random.randn(N, 1)
                #TODO
            else:
                raise ValueError("Invalid HiPPO type")
    
    return -np.array(mat)

In [ ]:
# Translated Legendre (LegT)
def build_LegT(n, k, lambda_n=1):
    A = None
    if lambda_n == 1:
        B = np.sqrt(2*n+1)
        A_base = np.sqrt(2*n+1) * np.sqrt(2*k+1)
        if k <= n:
            A = A_base * 1
        elif k >= n:
            A = A_base * np.power(-1, (n-k))
        else:
            raise ValueError("Invalid n and k")
            
    elif lambda_n == (np.sqrt(2*n+1) * np.power(-1, n)):
        B = (2*n+1) * np.power(-1, n)
        A_base = (2*n+1)
        if n <= k:
            A = A_base
        elif n >= k:
            A = A_base * np.power(-1, n-k)
        else:
            raise ValueError("Invalid n and k")
        
    return A, B

In [ ]:
# Translated Laguerre (LagT)
def build_LagT(alpha, beta, N):
    big_lambda = np.exp(.5 * (ss.gammaln(np.arange(N)+alpha+1) - ss.gammaln(np.arange(N)+1)))
    inverse_lambda = 1./big_lambda[:, None]
    pre_A = (np.eye(N) * ((1 + beta) / 2)) + np.tril(np.ones((N, N)), -1)
    pre_B = ss.binom(alpha + np.arange(N), np.arange(N))[:, None]
    
    A = -inverse_lambda * pre_A * big_lambda[None, :]
    B =  np.exp(-.5 * ss.gammaln(1-alpha)) * np.power(beta, (1-alpha)/2) * inverse_lambda * pre_B 
    return A, B

In [ ]:
#Scaled Legendre (LegS)
def build_LegS(n, k):
    A = None
    B = np.sqrt(2*n+1)
    if n > k:
        A = np.sqrt(2*n+1) * np.sqrt(2*k+1)
    elif n == k:
        A = n + 1
    else:
        A = np.zeros((n, n))
    return A, B

In [ ]:
# Translated Fourier
def build_FRU(N, theta=0.5):
    freqs = np.arange(N//2)
    d = np.stack([np.zeros(N//2), freqs], axis=-1).reshape(-1)[1:]
    pre_A = np.pi*(-np.diag(d, 1) + np.diag(d, -1))
    pre_B = np.zeros(N)
    pre_B[0::2] = 2**.5
    pre_B[0] = 1

    # Subtract off rank correction - this corresponds to the other endpoint u(t-1) in this case
    A = pre_A - pre_B[:, None] * pre_B[None, :]
    B = pre_B[:, None]
    return A, B